In [1]:
from line_solver import *
GlobalConstants.setVerbose(VerboseLevel.STD)

In [2]:
model = Network('model')

node = np.empty(4, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)
node[2] = Source(model,'Source')
node[3] = Sink(model,'Sink')

jobclass = OpenClass(model, 'Class1', 0)

node[0].setService(jobclass, HyperExp(0.5,3.0,10.0))
node[1].setService(jobclass, Exp(1))
node[2].setArrival(jobclass, Exp(0.1))

M = model.getNumberOfStations()
K = model.getNumberOfClasses()

P = model.initRoutingMatrix()
P.set(jobclass, jobclass, [[0,1,0,0],[0,0,0,1],[1,0,0,0],[0,0,0,0]])

model.link(P)

This example shows the execution of the solver on a 1-class 2-node open model.

In [3]:
#options = Solver.defaultOptions
#options.keep = True
#options.verbose = 1
#options.cutoff = 10
#options.seed = 23000
#options.iter_max = 200
#options.samples = 2000

# This part illustrates the execution of different solvers
# TODO: SSA result is buggy 
solver = np.array([], dtype=object)
#solver = np.append(solver, SolverCTMC(model,'keep',True))
solver = np.append(solver, SolverJMT(model,'seed',23000,'verbose',False,'keep',True))
solver = np.append(solver, SolverSSA(model,'seed',23000,'verbose',True,'samples',50000))
solver = np.append(solver, SolverFluid(model))
solver = np.append(solver, SolverMVA(model))
solver = np.append(solver, SolverNC(model))
#solver = np.append(solver, SolverMAM(model))
#solver = np.append(solver, LINE(model))

AvgTable = np.empty(len(solver), dtype=object)
for s in range(len(solver)):
    print(f'\nSOLVER: {solver[s].getName()}')
    AvgTable[s] = solver[s].getAvgTable()


SOLVER: SolverJMT
  Station JobClass      QLen      Util     RespT    ResidT      ArvR      Tput
0   Delay   Class1  0.022679  0.022679  0.214832  0.214832  0.099621  0.099607
1  Queue1   Class1  0.113320  0.102224  1.107529  1.107529  0.099607  0.100012
2  Source   Class1  0.000000  0.000000  0.000000  0.000000  0.000000  0.099621

SOLVER: SolverSSA
SSA samples:   50000
  Station JobClass      QLen      Util     RespT    ResidT      ArvR      Tput
0   Delay   Class1  0.021232  0.021232  0.213731  0.213731  0.100000  0.099338
1  Queue1   Class1  0.111210  0.099338  1.109886  1.109886  0.099338  0.100199
2  Source   Class1  0.000000  0.000000  0.000000  0.000000  0.000000  0.100000

SOLVER: SolverFluid
  Station JobClass      QLen      Util     RespT    ResidT  ArvR  Tput
0   Delay   Class1  0.021667  0.021667  0.216667  0.216667   0.0   0.1
1  Queue1   Class1  0.100000  0.100000  1.000000  1.000000   0.0   0.1
2  Source   Class1  0.000000  0.000000  0.000000  0.000000   0.0   0.1

SOL